In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt

import os
from utils import tools
from sklearn import preprocessing
from torchviz import make_dot, make_dot_from_trace


# import FlowNetC
from networks import FlowNetC

from PIL import Image
import numpy as np
import pandas as pd

import flowlib

from PIL import Image

import tqdm
import cv2
import csv
import time

In [10]:
class MyDataset:
    
    def __init__(self, num_train_img = 3400, base_dir = '/home/projectx/Documents/GitHub repos/VINET_modification/', sequence = 'Train_1/'):  # base_dir(image) and sequence(lstm) are directories
        self.base_dir = base_dir
        self.sequence = sequence
        self.base_path_img = self.base_dir + self.sequence
        
        self.image_files = os.listdir(self.base_path_img)
        self.image_files.sort()
        self.image_files = self.image_files[0:num_train_img]
        
        # normalization for lstm data
        self.train_scaler = preprocessing.StandardScaler()
        
        ## Omega.7 and load cells
        self.input_lstm = self.read_OMEGA7_LC()
        
        
        self.imu_seq_len = 20
    
    def read_OMEGA7_LC(self, path='data/result_1.csv'):
        # read csv data
        df = pd.read_csv(path,header = None)
        df = df[:874300]
        
        # take moving average of every 10 data points
        new_df = df.groupby(df.index//10).mean()
        array_input = np.array(new_df)
        
        # normalization
        array_input_scaled = self.train_scaler.fit_transform(array_input)
        
        input_lstm  = Variable(torch.from_numpy(array_input_scaled).type(torch.FloatTensor))
        input_lstm = input_lstm[:80000,:]
        
        
        
        # reshape to (num_dataset, sequence_length, feature_size)
        input_lstm = input_lstm.view(-1,20,11)
        
        return np.array(input_lstm)
    
    
    def get_input_lstm(self):
        return self.input_lstm
    
    def __len__(self):
        return len(self.image_files)
    
    def load_img_bat(self, idx, batch):
        batch_x = []
        batch_input_lstm = []
        for i in range(batch):
            x_data_np_1 = np.array(Image.open(self.base_path_img + self.image_files[idx + i]))
            x_data_np_2 = np.array(Image.open(self.base_path_img + self.image_files[idx+1 + i]))
            x_data_np_1 = x_data_np_1.reshape(3,512,448)
            x_data_np_2 = x_data_np_2.reshape(3,512,448)

#             ## 3 channels
#             x_data_np_1 = np.array([x_data_np_1, x_data_np_1, x_data_np_1])
#             x_data_np_2 = np.array([x_data_np_2, x_data_np_2, x_data_np_2])

            X = np.array([x_data_np_1, x_data_np_2])
            batch_x.append(X)
        
#           self.input_lstm of size: (num_dataset, sequence_length, feature_size)
            tmp = np.array(self.input_lstm[idx + i])
            batch_input_lstm.append(tmp)
            
        
        batch_x = np.array(batch_x)
        batch_input_lstm = np.array(batch_input_lstm)
        
        X = Variable(torch.from_numpy(batch_x).type(torch.FloatTensor).cuda())    
        X2 = Variable(torch.from_numpy(batch_input_lstm).type(torch.FloatTensor).cuda())    
        
        Y = X2[:,:,-3:]
        
     
        return X, X2 , Y.mean(1)

In [11]:
dataset = MyDataset()

/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
lstm_data = dataset.get_input_lstm()
# Though there are 4000, we will only call first 3400, constrained by the length of 'num_train_img'
lstm_data.shape

(4000, 20, 11)

In [13]:
def model_out_to_flow_png(output):
    out_np = output[0].data.cpu().numpy()

    #https://gitorchub.com/DediGadot/PatchBatch/blob/master/flowlib.py
    out_np = np.squeeze(out_np)
    out_np = np.moveaxis(out_np,0, -1)

    im_arr = flowlib.flow_to_image(out_np)
    im = Image.fromarray(im_arr)
    im.save('test.png')

In [14]:
import math
from networks.correlation_package.correlation import Correlation
from networks.submodules import *

In [15]:
cuda = torch.device('cuda')

In [16]:
model = FlowNetC.FlowNetC(batchNorm=False)
model.cuda();

In [17]:
dataset = MyDataset()
X, X2 , Y = dataset.load_img_bat(0,1)
X.size(), X2.size() , Y.size()

/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(torch.Size([1, 2, 3, 512, 448]), torch.Size([1, 20, 11]), torch.Size([1, 3]))

In [18]:
batch_size, timesteps, C, H, W = X.size()
img_pair = X.view(batch_size, timesteps * C, H, W)
# output = model(img_pair)

In [19]:
x1 = img_pair[:,0:3,:,:]
x2 = img_pair[:,3::,:,:]

# for x1
out_conv1a = model.conv1(x1)
out_conv2a = model.conv2(out_conv1a)
out_conv3a = model.conv3(out_conv2a)
x1.size(), x2.size(), out_conv3a.size()

(torch.Size([1, 3, 512, 448]),
 torch.Size([1, 3, 512, 448]),
 torch.Size([1, 256, 64, 56]))

In [20]:
# for x2
out_conv1b = model.conv1(x2)
out_conv2b = model.conv2(out_conv1b)
out_conv3b = model.conv3(out_conv2b)
out_conv3b.size()

torch.Size([1, 256, 64, 56])

In [21]:
# corrilation between x1 and x2
out_corr = model.corr(out_conv3a, out_conv3b)
out_corr.size()
# activate the corrilation
out_corr = model.corr_activation(out_corr)
out_corr.size()

/pytorch/torch/csrc/autograd/python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)


torch.Size([1, 441, 64, 56])

In [22]:
# Redirect top input stream and concatenate the features (addtional 32 features)
out_conv_redir = model.conv_redir(out_conv3a)
print(out_conv_redir.size())
in_conv3_1 = torch.cat((out_conv_redir,out_corr),1)
in_conv3_1.size()

torch.Size([1, 32, 64, 56])


torch.Size([1, 473, 64, 56])

In [23]:
# Merge conv layers
out_conv3_1 = model.conv3_1(in_conv3_1)
out_conv3_1.size()

torch.Size([1, 256, 64, 56])

In [24]:
out_conv4 = model.conv4_1(model.conv4(out_conv3_1))
out_conv4.size()

torch.Size([1, 512, 32, 28])

In [25]:
out_conv5 = model.conv5_1(model.conv5(out_conv4))
out_conv5.size()

torch.Size([1, 512, 16, 14])

In [26]:
out_conv6 = model.conv6_1(model.conv6(out_conv5))
out_conv6.size()

torch.Size([1, 1024, 8, 7])

In [27]:
# compute flow
flow6 = model.predict_flow6(out_conv6)
flow6.size()

torch.Size([1, 2, 8, 7])

In [28]:
flow6_up = model.upsampled_flow6_to_5(flow6)
flow6_up.size()

torch.Size([1, 2, 16, 14])

In [29]:
out_deconv5 = model.deconv5(out_conv6)
out_deconv5.size()

torch.Size([1, 512, 16, 14])

In [30]:
out_conv5.size(), out_deconv5.size(), flow6_up.size()

(torch.Size([1, 512, 16, 14]),
 torch.Size([1, 512, 16, 14]),
 torch.Size([1, 2, 16, 14]))

In [31]:
concat5 = torch.cat((out_conv5,out_deconv5,flow6_up),1)
concat5.size()

torch.Size([1, 1026, 16, 14])

In [32]:
flow5 = model.predict_flow5(concat5)
flow5.size()

torch.Size([1, 2, 16, 14])

In [33]:
flow5_up = model.upsampled_flow5_to_4(flow5)
flow5_up.size()

torch.Size([1, 2, 32, 28])

In [34]:
out_deconv4 = model.deconv4(concat5)
out_deconv4.size()

torch.Size([1, 256, 32, 28])

In [35]:
out_conv4.size(),out_deconv4.size(),flow5_up.size()

(torch.Size([1, 512, 32, 28]),
 torch.Size([1, 256, 32, 28]),
 torch.Size([1, 2, 32, 28]))

In [36]:
concat4 = torch.cat((out_conv4,out_deconv4,flow5_up),1)
concat4.size()

torch.Size([1, 770, 32, 28])

In [38]:
flow4       = model.predict_flow4(concat4)
flow4_up    = model.upsampled_flow4_to_3(flow4)
out_deconv3 = model.deconv3(concat4)
out_deconv3.size()

torch.Size([1, 128, 64, 56])

In [39]:
out_conv3_1.size(),out_deconv3.size(),flow4_up.size()

(torch.Size([1, 256, 64, 56]),
 torch.Size([1, 128, 64, 56]),
 torch.Size([1, 2, 64, 56]))

In [40]:
concat3 = torch.cat((out_conv3_1,out_deconv3,flow4_up),1)
concat3.size()

torch.Size([1, 386, 64, 56])

In [41]:
flow3       = model.predict_flow3(concat3)
flow3_up    = model.upsampled_flow3_to_2(flow3)
out_deconv2 = model.deconv2(concat3)
out_deconv2.size()

torch.Size([1, 64, 128, 112])

In [43]:
out_conv2a.size(),out_deconv2.size(),flow3_up.size()

(torch.Size([1, 128, 128, 112]),
 torch.Size([1, 64, 128, 112]),
 torch.Size([1, 2, 128, 112]))

In [42]:
concat2 = torch.cat((out_conv2a,out_deconv2,flow3_up),1)
concat2.size()

torch.Size([1, 194, 128, 112])

In [44]:
flow2 = model.predict_flow2(concat2)
flow2.size()

torch.Size([1, 2, 128, 112])

In [45]:
128*4, 112*4

(512, 448)

In [18]:
from networks import FlowNetC

In [19]:
model = FlowNetC.FlowNetC(batchNorm=False)
model.cuda();

In [ ]:
output = model(img_pair)

In [ ]:
output.size()

In [ ]:
import torch
import numpy as np
from networks.Fl